In [150]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path
import pyarrow

import importlib  
fa_etl = importlib.import_module("fa-etl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
county = "48203"

path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
# taxhist = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
# valhist = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/ValHist{county}.parquet")
# annual = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
# sales = pl.scan_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")
merged = pl.read_parquet(path_to_dir+f"dev/{county}/unified/merged.parquet")

In [162]:
merged.shape

(7539, 20)

In [163]:
merged.head(10)

PropertyID,Year,Value,AssessmentUsed,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,SaleAmt,TaxAmt,TaxAmtAdjusted
i64,i64,i64,str,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,f64
131825321,2015,12140,"""Assd""","""R""",2024-01-21,32.536926,-94.362986,"""501 HENLEY PER…","""MARSHALL""","""TX""","""75670""","""48203""","""020501""","""3004""","""B""","""48203_20501""",20000,22728,227.28
131825378,2015,78510,"""Assd""","""R""",2024-01-21,32.550108,-94.324417,"""3803 REDWOOD T…","""MARSHALL""","""TX""","""75672""","""48203""","""020301""","""3010""","""B""","""48203_20301""",114400,146983,1469.83
131825269,2018,128420,"""Assd""","""R""",2024-01-21,32.4687,-94.660496,"""1208 SAN SALVA…","""LONGVIEW""","""TX""","""75602""","""48203""","""020603""","""1020""","""B""","""48203_20603""",176100,233698,2336.98
131825309,2016,198510,"""Assd""","""R""",2024-01-21,32.468629,-94.659519,"""1214 SAN SALVA…","""LONGVIEW""","""TX""","""75602""","""48203""","""020603""","""1020""","""B""","""48203_20603""",228900,183317,1833.17
131825284,2018,115990,"""Assd""","""R""",2024-01-21,32.536153,-94.362573,"""505 SHIRLEY ST…","""MARSHALL""","""TX""","""75670""","""48203""","""020501""","""3000""","""B""","""48203_20501""",149900,257725,2577.25
131825387,2018,339850,"""Assd""","""R""",2024-01-21,32.499089,-94.574146,"""103 W CEDAR ST…","""HALLSVILLE""","""TX""","""75650""","""48203""","""020603""","""2028""","""B""","""48203_20603""",248900,750566,7505.66
131825407,2015,134150,"""Assd""","""R""",2024-01-21,32.46855,-94.663108,"""1110 SAN SALVA…","""LONGVIEW""","""TX""","""75602""","""48203""","""020603""","""1020""","""B""","""48203_20603""",93800,20511,205.11
131825383,2015,69090,"""Assd""","""R""",2024-01-21,32.498966,-94.576135,"""207 W CEDAR ST…","""HALLSVILLE""","""TX""","""75650""","""48203""","""020603""","""2031""","""B""","""48203_20603""",97800,112544,1125.44
131825306,2015,196080,"""Assd""","""R""",2024-01-21,32.468163,-94.65987,"""1211 CREEKWOOD…","""LONGVIEW""","""TX""","""75602""","""48203""","""020603""","""1020""","""B""","""48203_20603""",243800,180654,1806.54


In [164]:
merged.filter([
    pl.col("SaleAmt").is_not_null(),
    pl.col("Year") > 2017
]).select(
    ['PropertyID','Year','Value','AssessmentUsed','SaleAmt', 'TaxAmt', 'TaxAmtAdjusted']
).with_columns([
    (pl.col('TaxAmt')/pl.col('Value')).alias("ApproxTaxRate")
])

PropertyID,Year,Value,AssessmentUsed,SaleAmt,TaxAmt,TaxAmtAdjusted,ApproxTaxRate
i64,i64,i64,str,i64,i64,f64,f64
131825269,2018,128420,"""Assd""",176100,233698,2336.98,1.819794
131825284,2018,115990,"""Assd""",149900,257725,2577.25,2.221959
131825387,2018,339850,"""Assd""",248900,750566,7505.66,2.208521
131825382,2018,118390,"""Assd""",103200,263058,2630.58,2.221961
131825281,2019,159440,"""Assd""",221900,343905,3439.05,2.156956
131825183,2019,29590,"""Assd""",25000,63825,638.25,2.156979
131825516,2019,60070,"""Assd""",86500,74016,740.16,1.232162
131825090,2019,107120,"""Assd""",49000,153343,1533.43,1.431507
131825397,2019,161410,"""Assd""",182000,193965,1939.65,1.201691
